In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Input, Concatenate, Reshape
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')
from keras.models import load_model

Mounted at /content/drive


In [2]:
first_model = load_model('/content/drive/MyDrive/Election Project/first_model.h5')
df_2008 = pd.read_csv('/content/drive/MyDrive/Election Project/Complete 2008 updated.csv')

In [3]:
df_2008

,election_date,election_type,contest_status,constituency_number,constituency_name,province,assembly,voter_reg,validated_votes,votes_disq,candidateID,candidate_name,candidate_party,candidate_votes,candidate_share,candidate_rank,outcome
0,2/18/2008,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,387083,88325,629,NLFVkUN1Ih,Haji Ghulam Ahmed Bilour,Awami National Party,44210,0.500538,1,Win
1,2/18/2008,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,387083,88325,629,YDdR7JJToy,Ayub Shah,Pakistan Peoples Party Parliamentarians,37682,0.426629,2,Loss
2,2/18/2008,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,387083,88325,629,YjcqElajmi,Haji Abdul Jalil Jan,Jamiat Ulema-e Islam (Fazl),4103,0.046453,3,Loss
3,2/18/2008,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,387083,88325,629,yz2MD4YdZ4,Saifullah,Independents,661,0.007484,4,Loss
4,2/18/2008,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,387083,88325,629,ywaect63jO,Abdullah Jan,Independents,313,0.003544,5,Loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,12/28/2008,By-Election,Contested,NA-28,Buner,Khyber Paktunkhwa (NWFP),National,290460,72883,1805,ugGKWlS849,Molana Fazli Ghafoor,Pakistan Democratic Party,4701,0.064501,4,Loss
2276,12/28/2008,By-Election,Contested,NA-28,Buner,Khyber Paktunkhwa (NWFP),National,290460,72883,1805,62UAr3d7G4,Ali Sher Khan Advocate,Pakistan Muslim League (Nawaz),743,0.010194,5,Loss
2277,12/28/2008,By-Election,Contested,NA-28,Buner,Khyber Paktunkhwa (NWFP),National,290460,72883,1805,tpuww2nUIm,Nowshad Khan Advocate,Independents,732,0.010043,6,Loss
2278,12/28/2008,By-Election,Contested,NA-28,Buner,Khyber Paktunkhwa (NWFP),National,290460,72883,1805,ILoZ3baRF2,Molana Muhammad Hameed,Independents,195,0.002676,7,Loss


In [4]:
df_2008.isnull().sum()

election_date          0
election_type          0
contest_status         0
constituency_number    0
constituency_name      0
province               0
assembly               0
voter_reg              0
validated_votes        0
votes_disq             0
candidateID            0
candidate_name         0
candidate_party        0
candidate_votes        0
candidate_share        0
candidate_rank         0
outcome                0
dtype: int64

In [7]:
x=df_2008['constituency_number'].unique()
len(df_2008['constituency_number'].unique())

269

In [8]:
x

array(['NA-1', 'NA-10', 'NA-100', 'NA-101', 'NA-102', 'NA-103', 'NA-104',
       'NA-105', 'NA-106', 'NA-107', 'NA-108', 'NA-109', 'NA-11',
       'NA-110', 'NA-111', 'NA-112', 'NA-113', 'NA-114', 'NA-115',
       'NA-116', 'NA-117', 'NA-118', 'NA-12', 'NA-120', 'NA-121',
       'NA-122', 'NA-123', 'NA-124', 'NA-125', 'NA-126', 'NA-127',
       'NA-128', 'NA-129', 'NA-13', 'NA-130', 'NA-131', 'NA-132',
       'NA-133', 'NA-134', 'NA-135', 'NA-136', 'NA-137', 'NA-138',
       'NA-139', 'NA-14', 'NA-140', 'NA-141', 'NA-142', 'NA-143',
       'NA-144', 'NA-145', 'NA-146', 'NA-147', 'NA-148', 'NA-149',
       'NA-15', 'NA-150', 'NA-151', 'NA-152', 'NA-153', 'NA-154',
       'NA-155', 'NA-156', 'NA-157', 'NA-158', 'NA-159', 'NA-16',
       'NA-160', 'NA-161', 'NA-162', 'NA-163', 'NA-164', 'NA-165',
       'NA-166', 'NA-167', 'NA-168', 'NA-169', 'NA-17', 'NA-170',
       'NA-171', 'NA-172', 'NA-173', 'NA-174', 'NA-175', 'NA-176',
       'NA-177', 'NA-178', 'NA-179', 'NA-18', 'NA-180', 'NA-18

In [ ]:
'''df_2008 = df_2008.dropna(subset=['constituency_number'])
exclude_prefixes = ['PP', 'PB', 'PK', 'PS']
df_2008 = df_2008[~df_2008['constituency_number'].str.startswith(tuple(exclude_prefixes))]'''

"df_2008 = df_2008.dropna(subset=['constituency_number'])\nexclude_prefixes = ['PP', 'PB', 'PK', 'PS']\ndf_2008 = df_2008[~df_2008['constituency_number'].str.startswith(tuple(exclude_prefixes))]"

In [ ]:
'''df_2008['voter_reg'].fillna(df_2008['voter_reg'].mean(), inplace=True)
df_2008['validated_votes'].fillna(df_2008['validated_votes'].mean(), inplace=True)
df_2008['votes_disq'].fillna(df_2008['votes_disq'].mean(), inplace=True)
df_2008['candidate_votes'].fillna(df_2008['candidate_votes'].mean(), inplace=True)
df_2008['candidate_share'].fillna(df_2008['candidate_share'].mean(), inplace=True)
df_2008['candidate_rank'].fillna(df_2008['candidate_rank'].mean(), inplace=True)'''

"df_2008['voter_reg'].fillna(df_2008['voter_reg'].mean(), inplace=True)\ndf_2008['validated_votes'].fillna(df_2008['validated_votes'].mean(), inplace=True)\ndf_2008['votes_disq'].fillna(df_2008['votes_disq'].mean(), inplace=True)\ndf_2008['candidate_votes'].fillna(df_2008['candidate_votes'].mean(), inplace=True)\ndf_2008['candidate_share'].fillna(df_2008['candidate_share'].mean(), inplace=True)\ndf_2008['candidate_rank'].fillna(df_2008['candidate_rank'].mean(), inplace=True)"

In [ ]:
df_2008.isnull().sum()

election_date          0
election_type          0
contest_status         0
constituency_number    0
constituency_name      0
province               0
assembly               0
voter_reg              0
validated_votes        0
votes_disq             0
candidateID            0
candidate_name         0
candidate_party        0
candidate_votes        0
candidate_share        0
candidate_rank         0
outcome                0
dtype: int64

In [ ]:
desire_col = ['constituency_number', 'province', 'candidate_name','candidate_party','candidate_votes','candidate_share','outcome']
data = df_2008[desire_col]

In [ ]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [ ]:
def tokenize_and_pad(data, max_len):
    sequences = pad_sequences(data, maxlen=max_len, padding='post')
    return sequences.tolist()

In [ ]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-62-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-62-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [ ]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'candidate_votes', 'candidate_share']]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [ ]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [ ]:
# Assuming X_train and y_train are your training data (adjust this based on your actual data)
X_train, X_test, y_train, y_test = train_test_split(
    processed_data[['province', 'candidate_name_padded', 'candidate_party_padded', 'candidate_votes', 'candidate_share']],
    processed_data['outcome'],
    test_size=0.2,
    random_state=42
)

In [ ]:
from keras.layers import Bidirectional, Reshape

model2 = Sequential()
model2.add(first_model)
model2.add(Reshape((-1, 1)))
model2.add(LSTM(64, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(32, return_sequences=True))
model2.add(LSTM(32))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

# Train the model
model2.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)
model2.save('/content/drive/MyDrive/Election Project/second_model.h5')
model2.summary()

Epoch 1/10
46/46 [==============================] - 21s 157ms/step - loss: 0.6346 - accuracy: 0.8568 - val_loss: 0.4986 - val_accuracy: 0.9123
Epoch 2/10
46/46 [==============================] - 5s 108ms/step - loss: 0.4222 - accuracy: 0.8684 - val_loss: 0.3103 - val_accuracy: 0.9123
Epoch 3/10
46/46 [==============================] - 4s 82ms/step - loss: 0.3905 - accuracy: 0.8684 - val_loss: 0.3013 - val_accuracy: 0.9123
Epoch 4/10
46/46 [==============================] - 4s 80ms/step - loss: 0.3923 - accuracy: 0.8684 - val_loss: 0.3028 - val_accuracy: 0.9123
Epoch 5/10
46/46 [==============================] - 5s 118ms/step - loss: 0.3912 - accuracy: 0.8684 - val_loss: 0.3073 - val_accuracy: 0.9123
Epoch 6/10
46/46 [==============================] - 5s 98ms/step - loss: 0.3923 - accuracy: 0.8684 - val_loss: 0.3113 - val_accuracy: 0.9123
Epoch 7/10
46/46 [==============================] - 4s 92ms/step - loss: 0.3908 - accuracy: 0.8684 - val_loss: 0.3192 - val_accuracy: 0.9123
Epoch 8/1

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model2.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

15/15 [==============================] - 0s 20ms/step - loss: 0.3584 - accuracy: 0.8860
Test Loss: 0.3583723306655884
Test Accuracy: 0.8859649300575256


In [ ]:
predictions_2008 = model2.predict(X_test_padded)
predictions_2008

15/15 [==============================] - 3s 19ms/step


array([[0.14242177],
       [0.1424218 ],
       [0.1424218 ],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.1424218 ],
       [0.1424218 ],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.1424218 ],
       [0.14242177],
       [0.1424218 ],
       [0.14242177],
       [0.1424218 ],
       [0.1424218 ],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242178],
       [0.14242178],
       [0.1424218 ],
       [0.14242177],
       [0.1424218 ],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.14242177],
       [0.1424218 ],
       [0.14242177],
       [0.142